In [24]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [2]:
! wget "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

--2024-06-08 06:54:43--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-06-08 06:54:43 (26.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
text = open("input.txt").read()
len(text)

1115394

In [5]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:200])

torch.Size([1115394]) torch.int64


In [9]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

In [14]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

xb, yb = get_batch('train')
xb.shape, yb.shape, xb, yb

(torch.Size([128, 256]),
 torch.Size([128, 256]),
 tensor([[ 1, 63, 53,  ..., 46, 39, 57],
         [ 1, 46, 39,  ...,  1, 46, 47],
         [46, 63,  6,  ...,  0, 19, 24],
         ...,
         [53, 50, 42,  ..., 52, 41, 43],
         [56,  6,  1,  ..., 46, 39, 42],
         [43, 42,  1,  ..., 53, 56,  1]], device='cuda:0'),
 tensor([[63, 53, 59,  ..., 39, 57, 58],
         [46, 39, 57,  ..., 46, 47, 51],
         [63,  6,  1,  ..., 19, 24, 27],
         ...,
         [50, 42,  1,  ..., 41, 43, 56],
         [ 6,  1, 51,  ..., 39, 42,  1],
         [42,  1, 41,  ..., 56,  1, 57]], device='cuda:0'))

# Transformers Modules

In [15]:
# class Head(nn.Module):
#     def __init__(self, head_size):
#         super().__init__()
#         self.key = nn.Linear(dmodel, head_size, bias = False)
#         self.query = nn.Linear(dmodel, head_size, bias = False)
#         self.value = nn.Linear(dmodel, head_size, bias = False)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         B, T, C = x.shape
#         k = self.key(x)
#         q = self.query(x)
#         wei = q @ k.transpose(-2,-1) * C**-0.5
#         wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
#         wei = F.softmax(wei, dim=-1)
#         wei = self.dropout(wei)
#         v = self.value(x)
#         out = wei @ v
#         return out

# class MultiHeadAttentionV1(nn.Module):
#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.proj = nn.Linear(dmodel, dmodel)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         out = torch.cat([h(x) for h in self.heads], dim=-1)
#         out =  self.dropout(self.proj(out))
#         return out

In [25]:
class MultiHeadAttentionV2(nn.Module):
    def __init__(self, num_heads, dmodel):
        super().__init__()
        self.num_heads = num_heads
        self.dropout = nn.Dropout(dropout)
        self.kqv = nn.Linear(dmodel, 3*dmodel, bias = False)
        self.proj = nn.Linear(dmodel, dmodel)
        self.attn_dropout = nn.Dropout(dropout)
        self.res_dropout = nn.Dropout(dropout)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))


    def forward(self, x):
        B, T, C = x.size()
        h = self.num_heads
        hs = C//h
        k, q, v = self.kqv(x).split(dmodel, dim=2)

        k = k.view(B, T, h, hs).transpose(1,2) # (B, h, T, C/h)
        q = q.view(B, T, h, hs).transpose(1,2) # (B, h, T, C/h)
        v = v.view(B, T, h, hs).transpose(1,2) # (B, h, T, C/h)

        attn = (q @ k.transpose(-2,-1))/math.sqrt(hs) # (B, h, T, T)

        attn = attn.masked_fill(self.tril[:, :, :T, :T]==0, float('-inf'))

        attn = F.softmax(attn, dim=-1)

        attn = self.attn_dropout(attn)
        out = attn @ v # (B, h, T, C/h)

        out = out.transpose(1,2).contiguous().view(B, T, C)
        out =  self.res_dropout(self.proj(out))
        return out

In [26]:
class FeedForward(nn.Module):
    def __init__(self, dmodel):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dmodel, dmodel*4),
            nn.ReLU(),
            nn.Linear(dmodel*4, dmodel),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [27]:
class Block(nn.Module):
    def __init__(self, dmodel, n_head):
        super().__init__()
        self.sa = MultiHeadAttentionV2(n_head, dmodel)
        self.ffwd = FeedForward(dmodel)
        self.ln1 = nn.LayerNorm(dmodel)
        self.ln2 = nn.LayerNorm(dmodel)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # (B, T, C)
        x = x + self.ffwd(self.ln2(x)) # (B, T, C) position wise
        return x

In [28]:
# bigram

class GPT(nn.Module):
    def __init__(self, dmodel, n_head, n_layer):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, dmodel)
        self.pos_embedding_table = nn.Embedding(block_size, dmodel)
        self.blocks = nn.Sequential(*[ Block(dmodel, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(dmodel)
        self.lm_head = nn.Linear(dmodel, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.pos_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, V)

        loss = None

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # unpacking the batches of embeddings
            targets = targets.view(B*T) # unpacking the batches of targets
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, 1)
            idx_next = torch.multinomial(probs, 1)
            print(decode(idx_next[0].tolist()), end="")
            idx = torch.cat((idx, idx_next), 1)
        return idx

In [34]:
dmodel=300
batch_size = 64
block_size = 128
n_head = 6
n_layer = 6
dropout = 0.2
lr = 5e-4

In [35]:
model = GPT(dmodel, n_head, n_layer)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


In [36]:
%%time
epochs = 10000
et = epochs/10
model.train()
for epoch in range(epochs):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % et == 0:
        print(f'{epoch} || loss: {loss:.4f}')

0 || loss: 4.3887
1000 || loss: 1.5346
2000 || loss: 1.3920
3000 || loss: 1.3366
4000 || loss: 1.3128
5000 || loss: 1.2114
6000 || loss: 1.2360
7000 || loss: 1.1610
8000 || loss: 1.1345
9000 || loss: 1.0998
CPU times: user 25min 11s, sys: 2.77 s, total: 25min 13s
Wall time: 25min 21s


In [43]:
with torch.no_grad():
    model.eval()
    context = torch.tensor([[0]], device = device)
    out = model.generate(context, 2000)



MENENIUS:
Yea,'tis for proceed.

CORIOLANUS:
Let them like sure out.

SICINIUS:
We shall have attended their beachs.

BRUTUS:
You are enough new so little at you;
Where'er the gates, you are men as as see,
As you give the highway to noble fin's stand.

GLOUCESTER:
He does and heavenly to Tarpeian lords.

BY:
Now, noble lord, I warrant, if we saw in flaughth Grumio,
Provided them both; therefore tell the world:
What sail will I am your house to-morrow?

KING RICHARD III:
But stay, now I am dead.

QUEEN ELIZABETH:
Shall I gg, or worthy my face I be Leontes!

ROMEO:
By this marriage renown'd forth, for such charge
Gave me to death, I bid my park about me,
Before I tell my cousin, and my doubt,
The other tribunes shall suffer scall my brother,
But now have any more grace, than the many limbs;
Whose modest length of your own remedies.
I came from what advancement from the crown,
Say is writ from your blood and with shrife attempt
And bid the crown thine intelligence, grave's flack
This gef

In [40]:
torch.save(model.state_dict(), 'tiny_shakespearev2.pt')

In [ ]:

# model = GPT()
# state_dict = torch.load('tiny_shakespearev2.pt', map_location=device)
# model.load_state_dict(state_dict)
# model.to(device);

In [ ]:
# # inference
# with torch.no_grad():
#     model.eval()
#     context = torch.tensor([[0]], device = device)
#     out = model.generate(context, 200000)
#     # print(decode(out[0].tolist()))
